<a href="https://colab.research.google.com/github/AndrewMayneProjects/Whisper/blob/main/WhisperVideoDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're looking at this on GitHub and new to Python Notebooks or Colab, click the Google Colab badge above 👆

#📼 OpenAI Whisper + Google Drive Video Transcription

###This application will extract audio from all the video files in a Google Drive folder and create a high-quality transcription with OpenAI's Whisper automatic speech recognition system.

*Note: This requires giving the application permission to connect to your drive. Only you will have access to the contents of your drive, but please read the warnings carefully.*

This notebook application:
1. Connects to your Google Drive when you give it permission.
2. Creates a WhisperVideo folder and three subfolders (ProcessedVideo, AudioFiles and TextFiles.)
3. When you run the application it will search for all the video files (.mp4, .mov, mkv and .avi) in your WhisperVideo folder, transcribe them and then move the file to WhisperVideo/ProcessedVideo and save the transcripts to WhisperVideo/TextFiles. It will also add a copy of the new audio file to WhisperVideo/AudioFiles

###**For faster performance set your runtime to "GPU"**
*Click on "Runtime" in the menu and click "Change runtime type". Select "GPU".*


**Note: If you add a new file after running this application you'll need to remount the drive in step 1 to make them searchable**

##1. Load the code libraries

In [ ]:
!pip install youtube_dl
!pip install git+https://github.com/openai/whisper.git 
!sudo apt update && sudo apt install ffmpeg
!pip install librosa

import whisper
import time
import librosa
import soundfile as sf
import re
import os

# model = whisper.load_model("tiny.en")
model = whisper.load_model("base.en")  # load the base model
# model = whisper.load_model("small.en")
# model = whisper.load_model("medium.en")
# model = whisper.load_model("large")

##2. Give the application permission to mount the drive and create the folders

In [ ]:
# Create the Drive folders
from google.colab import drive
drive.mount("/content/drive", force_remount=True) # This will prompt for authorization.

# This will create the WhisperVideo files if they don't exist.
folders =  ["WhisperVideo/", "WhisperVideo/ProcessedVideo/", "WhisperVideo/TextFiles/", "WhisperVideo/AudioFiles/"]
for folder in folders:
  path = "/content/drive/MyDrive/" + folder
  if not os.path.exists(path): # Create the folder if it does not exist
    os.mkdir(path) 


Mounted at /content/drive


##3. Upload any audio files you want transcribed in the "WhisperVideo" folder in your Google Drive.

##4. Extract audio from the video files and create a transcription

In [ ]:
# Load all the audio file paths in a Google Drive folder
from google.colab import drive
drive.mount("/content/drive", force_remount=True) # This will prompt for authorization.

# Get the list of video files from the WhisperVideo folder
video_files = os.listdir("/content/drive/MyDrive/WhisperVideo/")

# Loop through the video files and transcribe them
for video_file in video_files:

  # Skip the file if it is not a video format
  if not video_file.endswith((".mp4", ".mov", ".avi", ".mkv")):
    continue

  # Extract the audio from the video file using librosa
  video_path = "/content/drive/MyDrive/WhisperVideo/" + video_file
  audio_path = "/content/drive/MyDrive/WhisperVideo/AudioFiles/" + video_file[:-4] + ".wav" # Replace the video extension with .wav


  y, sr = librosa.load(video_path, sr=16000) # Load the audio with 16 kHz sampling rate
  sf.write(audio_path, y, sr) # Save the audio as a wav file

  # Transcribe the audio file using Whisper
  result = model.transcribe(audio_path)
  text = result["text"].strip()
  text = text.replace(". ", ".\n\n")

  # Save the transcription as a text file in Google Docs
  text_file = video_file[:-4] + ".txt" # Replace the video extension with .txt
  text_path = "/content/drive/MyDrive/WhisperVideo/TextFiles/" + text_file
  with open(text_path, "w") as f:
    f.write(text)

  # Move the video file to the ProcessedVideo folder
  processed_path = "/content/drive/MyDrive/WhisperVideo/ProcessedVideo/" + video_file
  os.rename(video_path, processed_path)

  # Print a message to indicate the progress
  print(f"Processed {video_file} and saved the transcription as {text_file}")